##### Copyright 2024 Google LLC.

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Building a RAG using Gemma with Elasticsearch, Ollama and Langchain

This tutorial will guide you through building a **Retrieval-Augmented Generation (RAG)** application using the **Gemma 2 9B** model, **LangChain**, **Ollama**, and **Elasticsearch**. You'll go through each step in detail, ensuring that even if you're new to RAGs or Large Language Models (LLMs), you'll be able to follow along and build your own local AI application.

## Introduction

**Retrieval-Augmented Generation (RAG)** is a technique that combines large language models (LLMs) with external knowledge sources to generate more accurate and contextually relevant responses. It involves two main components:

* **Retriever**: Based on the user's query, the retriever fetches relevant documents from a dataset to provide additional context to the LLM.

* **Generator**: The LLM uses the retrieved context along with the user's query to generate accurate and coherent responses.

By combining retrieval with generation, RAG systems can produce responses that are both informed and coherent, making them ideal for tasks like question answering over custom datasets.

[**Gemma**](https://ai.google.dev/gemma) is a family of lightweight, state-of-the-art open language models from Google. Built from the same research and technology used to create the Gemini models, Gemma models are text-to-text, decoder-only large language models (LLMs) available in English, with open weights, pre-trained variants, and instruction-tuned variants.

The **Gemma 2 9B IT Q6 K** model is a quantized instruction-tuned version of the Gemma model, optimized for performance while reducing computational load. This makes it possible to deploy the model in environments with limited resources, such as a laptop, desktop, or your own cloud infrastructure. It democratizes access to state-of-the-art AI models and fosters innovation for everyone.

[**LangChain**](https://python.langchain.com/) is a framework for developing applications powered by language models. It provides a suite of tools and integrations that simplify building complex AI applications, such as chatbots, question-answering systems, and more. LangChain allows you to chain together various components like prompts, LLMs, and retrievers to create sophisticated pipelines.

[**Ollama**](https://ollama.ai/) is a tool that simplifies running language models locally. It allows you to manage and serve multiple models efficiently, making it easier to deploy and test AI models on your machine. With Ollama, you can switch between different models and versions seamlessly, providing flexibility in development and experimentation. You can browse the available Gemma 2 models at the [Ollama Gemma 2 Model Catalog](https://ollama.com/library/gemma2).

[**Elasticsearch**](https://www.elastic.co/elasticsearch/) is a powerful open-source search and analytics engine. It allows you to store, search, and analyze large volumes of data quickly and in near real-time. In this tutorial, Elasticsearch serves as the data source and vector store for our RAG application, enabling efficient retrieval of relevant documents based on user queries.

By combining these tools, you can build a local RAG application that leverages the strengths of each component to create a powerful AI application capable of handling tasks like question answering over custom datasets—all running locally on a modest GPU like the T4.

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/Gemma/Using_Gemma_with_Elasticsearch_and_LangChain.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

## Setup

Before you begin, make sure you have a Google Colab account.

### Select the Colab Runtime

First, you'll need to set up your Google Colab environment:

1. **Open Google Colab** and create a new notebook.
2. In the upper-right corner of the Colab window, click on the **▾ (Additional connection options)** button.
3. Select **Change runtime type**.
4. Under **Hardware accelerator**, choose **GPU**.
5. Ensure that the **GPU type** is set to **T4**.

This will provide sufficient resources to run the Gemma 2 9B model.

### Gemma Setup

Before diving into the tutorial, let's set up Gemma:

1. **Create a Hugging Face Account**: If you don't have one, you can sign up for a free account [here](https://huggingface.co/join).
2. **Access the Gemma Model**: Visit the [Gemma model page](https://huggingface.co/collections/google/gemma-2-release-667d6600fd5220e7b967f315) and accept the usage conditions.
3. **Generate a Hugging Face Token**: Go to your Hugging Face [settings page](https://huggingface.co/settings/tokens) and generate a new access token (preferably with `write` permissions).

**Once you've completed these steps, you're ready to move on to the next section where you'll set up environment variables in your Colab environment.**

### Configure Your Credentials


Next, we'll securely store your Hugging Face token using the Colab Secrets manager:

1. Open your Google Colab notebook and click on the 🔑 Secrets tab in the left panel. <img src="https://storage.googleapis.com/generativeai-downloads/images/secrets.jpg" alt="The Secrets tab is found on the left panel." width=50%>
2. **Add Hugging Face Token**:
   - Create a new secret named `HF_TOKEN`.
   - Paste your Hugging Face token into the Value input box.
   - Toggle the button to allow notebook access to the secret.

Now, set the environment variables in your notebook:

In [2]:
import os
from google.colab import userdata

# Set Hugging Face token
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

This code retrieves your secrets and sets them as environment variables, which you will use later in the tutorial.

### Installing Dependencies


Next, you need to install all the required dependencies. Run the following cell to install them:


In [3]:
!pip install -q langchain tiktoken
!pip install -q langchainhub langchain-huggingface langchain-text-splitters
!pip install -q sentence-transformers==2.2.2
!pip install -q -U huggingface-hub
!pip install -q -U elasticsearch==8.15.1 langchain-elasticsearch==0.3.0
!pip install -q langchain-ollama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: pip

### Import Dependencies

In [4]:
import os
import time
from google.colab import userdata
from typing import Dict

from elasticsearch import Elasticsearch

from langchain_ollama.chat_models import ChatOllama
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_elasticsearch import ElasticsearchStore, ElasticsearchRetriever
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import hub
from langchain_core.output_parsers import BaseTransformOutputParser
from langchain_core.runnables import RunnablePassthrough

## Gemma

Gemma models are designed to be lightweight yet powerful, making them suitable for environments with limited resources. They support various text generation tasks and are available in instruction-tuned variants, which means they've been trained to follow instructions provided in prompts.

#### Prompt Formatting

Instruction-tuned models use specific control tokens to format prompts:

- **`user`**: Indicates a user turn.
- **`model`**: Indicates a model turn.
- **`<start_of_turn>`**: Marks the beginning of a dialogue turn.
- **`<end_of_turn>`**: Marks the end of a dialogue turn.

This formatting helps the model understand and generate conversational responses. Refer to the [official documentation](https://ai.google.dev/gemma/docs/formatting) for more details.

### Installing and Running Ollama

You will use Ollama to run the Gemma model locally.

First, install Ollama by running:

In [5]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


Then, start the Ollama server in the background.


In [6]:
!nohup ollama serve > ollama.log &

nohup: redirecting stderr to stdout


Ollama provides a library of pre-configured models, including Gemma 2 models. You can browse the available Gemma 2 models at the [Ollama Gemma 2 Model Catalog](https://ollama.com/library/gemma2). This allows you to switch between different Gemma 2 models easily.

In this notebook, you'll use the [gemma2:9b-instruct-q6_K](https://ollama.com/library/gemma2:9b-instruct-q6_K) model.


In [7]:
!ollama run gemma2:9b-instruct-q6_K "What is the capital of France?" 2> ollama.log

The capital of France is **Paris**. 🇫🇷  




You should see the model's response in the output.

### Integrate Gemma with LangChain

Now, let's set up the Gemma model with LangChain:

In [8]:
llm = ChatOllama(
    model="gemma2:9b-instruct-q6_K",
    temperature=0.8
)

Test the LLM by asking a simple question:


In [9]:
response = llm.invoke("What is the capital of France?")
print(response.content)

The capital of France is **Paris**. 🇫🇷  



## Setting up Elasticsearch

Next, you'll set up a local instance of Elasticsearch to serve as our data source and vector store.

**Note:** You're choosing to run Elasticsearch locally instead of using [Elastic Cloud](https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud) to keep the tutorial self-contained and to avoid external dependencies. This way, you can run everything on your machine without needing internet access or incurring any cloud costs.


### Download and Install Elasticsearch

First, you need to download and install Elasticsearch.


In [10]:
# Removes any previous Elasticsearch installations:
!rm -rf elasticsearch*

Download Elasticsearch Version 8.15.1, extract the archive and set permissions.


In [11]:
ESVERSION = "8.15.1"

In [ ]:
%%bash -s "$ESVERSION"
export ESVERSION=$1

# Download and extract ES
wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-${ESVERSION}-linux-x86_64.tar.gz
wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-${ESVERSION}-linux-x86_64.tar.gz.sha512
tar -xzf elasticsearch-${ESVERSION}-linux-x86_64.tar.gz

# The binary's integrity is verified using SHA-512
shasum -a 512 -c elasticsearch-${ESVERSION}-linux-x86_64.tar.gz.sha512

# Set up user to run ES daemon and configure cgroups
umount /sys/fs/cgroup
apt install cgroup-tools
sudo chown -R daemon:daemon elasticsearch-${ESVERSION}/

### Configure Elasticsearch

For demonstration purposes, let's disable security settings.   
**Note**: In a production environment, always enable security features.

Open the Elasticsearch configuration file and append the following settings:

In [13]:
with open(f'./elasticsearch-{ESVERSION}/config/elasticsearch.yml', 'a') as f:
    f.write("xpack.security.enabled: false\n")
    f.write("xpack.security.authc:\n")
    f.write("  anonymous:\n")
    f.write("    username: anonymous_user\n")
    f.write("    roles: superuser\n")
    f.write("    authz_exception: true\n")

If you want to verify that the **elasticsearch.yml** file is written correctly, you can uncomment and run the following code block.

In [14]:
# with open(f'./elasticsearch-{ESVERSION}/config/elasticsearch.yml', 'r') as f:
#     print(f.read())

### Run Elasticsearch

Now, let's start Elasticsearch as a daemon process:


In [15]:
%%bash --bg -s "$ESVERSION"

export ESVERSION=$1

sudo -H -u daemon elasticsearch-${ESVERSION}/bin/elasticsearch

It takes Elasticsearch a while to get running, so be sure to wait a few seconds. You can run a manual 60-second sleep command to ensure Elasticsearch has enough time to start:

In [16]:
time.sleep(60)

Once the instance has been started, you can check if Elasticsearch is running by listing the processes. You should see several elasticsearch processes running.

In [17]:
!ps -ef | grep elastic

root        3166    3164  0 22:36 ?        00:00:00 sudo -H -u daemon elasticsearch-8.15.1/bin/elast
daemon      3167    3166  8 22:36 ?        00:00:05 /content/elasticsearch-8.15.1/jdk/bin/java -Xms4
daemon      3251    3167 99 22:36 ?        00:00:56 /content/elasticsearch-8.15.1/jdk/bin/java -Des.
daemon      3305    3251  0 22:36 ?        00:00:00 /content/elasticsearch-8.15.1/modules/x-pack-ml/
root        3552     908  0 22:37 ?        00:00:00 /bin/bash -c ps -ef | grep elastic
root        3554    3552  0 22:37 ?        00:00:00 grep elastic


Verify Elasticsearch is running by making a request to the cluster. Here, you use the default elastic superuser and password password to initialize the cluster so that you can perform anonymous calls moving forward.

**WARNING**: Do not pass user passwords like this in real life.

In [18]:
!curl -u elastic:password -H 'Content-Type: application/json' -XGET http://localhost:9200/?pretty=true

{
  "name" : "2583c25f4c9e",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "cn_3_5SURQOcQB2BVm-37w",
  "version" : {
    "number" : "8.15.1",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "253e8544a65ad44581194068936f2a5d57c2c051",
    "build_date" : "2024-09-02T22:04:47.310170297Z",
    "build_snapshot" : false,
    "lucene_version" : "9.11.1",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


## QA with RAG Using Elasticsearch

Now, you'll perform question answering using Retrieval-Augmented Generation (RAG) with Elasticsearch by implementing the two stages in a RAG-based architecture:

1. **Retrieval**: Retrieves relevant context based on the user's query.
2. **Generation**: Uses the LLM to generate answers using the retrieved context.

### Retrieval

In this stage, you will perform the following steps:

* **Create a sample dataset**: Use sample Pokémon data.
* **Preparing Documents for Indexing**: Split documents into manageable chunks.
* **Create Embeddings of the Data**: Convert text data into numerical vectors.
* **Store the Embeddings in the Vector Store**: Index the embeddings in Elasticsearch.
* **Create a Retriever**: Set up a retriever to fetch relevant documents.


Initialize the Elasticsearch client by connecting to the local instance.


In [19]:
es_url = "http://localhost:9200"
client = Elasticsearch(hosts=[es_url])

# Verify connection
if client.ping():
    print("Connected to Elasticsearch")
else:
    print("Could not connect to Elasticsearch")

print(client.info())

Connected to Elasticsearch
{'name': '2583c25f4c9e', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'cn_3_5SURQOcQB2BVm-37w', 'version': {'number': '8.15.1', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': '253e8544a65ad44581194068936f2a5d57c2c051', 'build_date': '2024-09-02T22:04:47.310170297Z', 'build_snapshot': False, 'lucene_version': '9.11.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


#### Create a sample dataset

First, create some sample data to index. To do this, let's use descriptions of various Pokémon:

In [20]:
data = [
    {
        "name": "Bulbasaur",
        "description": "Bulbasaur has a strange seed planted on its back at birth. The plant sprouts and grows with Bulbasaur.",
        "type": "Grass/Poison"
    },
    {
        "name": "Charmander",
        "description": "Charmander obviously prefers hot places. When it rains, steam is said to spout from the tip of Charmander's tail.",
        "type": "Fire"
    },
    {
        "name": "Squirtle",
        "description": "After birth, Squirtle's back swells and hardens into a shell. Squirtle powerfully sprays foam from its mouth.",
        "type": "Water"
    },
    {
        "name": "Pikachu",
        "description": "When several Pikachu gather, their electricity could build and cause lightning storms.",
        "type": "Electric"
    },
    {
        "name": "Jigglypuff",
        "description": "When Jigglypuff sings, it never pauses to breathe. If Jigglypuff is in battle against an opponent that does not easily fall asleep, it cannot breathe, endangering its life.",
        "type": "Normal/Fairy"
    },
    {
        "name": "Meowth",
        "description": "Meowth adores round objects. It wanders the streets on a nightly basis to look for dropped loose change.",
        "type": "Normal"
    },
    {
        "name": "Psyduck",
        "description": "While lulling its enemies with its vacant look, this wily Psyduck will use psychokinetic powers.",
        "type": "Water"
    },
    {
        "name": "Mewtwo",
        "description": "Mewtwo was created by a scientist after years of horrific gene splicing and DNA engineering experiments.",
        "type": "Psychic"
    },
    {
        "name": "Snorlax",
        "description": "Snorlax's daily routine consists of nothing more than eating and sleeping. It is such a docile Pokémon that children use Snorlax's expansive belly as a place to play.",
        "type": "Normal"
    },
    {
        "name": "Gengar",
        "description": "Sometimes, on a dark night, your shadow thrown by a streetlight will suddenly and startlingly overtake you. It is actually a Gengar running past you.",
        "type": "Ghost/Poison"
    },
    {
        "name": "Lapras",
        "description": "People have driven Lapras almost to the point of extinction. In the evenings, Lapras is said to sing plaintively as it seeks what few others of its kind still remain.",
        "type": "Water/Ice"
    },
    {
        "name": "Dragonite",
        "description": "Dragonite is capable of circling the globe in just sixteen hours. It is a kindhearted Pokémon that leads lost ships in a storm to the safety of land.",
        "type": "Dragon/Flying"
    },
    {
        "name": "Ditto",
        "description": "Ditto rearranges its cell structure to transform itself into other shapes. However, if Ditto tries to transform itself by relying on its memory, it may get details wrong.",
        "type": "Normal"
    },
    {
        "name": "Magikarp",
        "description": "Magikarp is a pathetic excuse for a Pokémon that is only capable of flopping and splashing. This behavior prompted scientists to undertake research into Magikarp.",
        "type": "Water"
    },
    {
        "name": "Charizard",
        "description": "Charizard flies around the sky in search of powerful opponents. Charizard breathes fire of such great heat that it melts anything.",
        "type": "Fire/Flying"
    },
    {
        "name": "Onix",
        "description": "Onix burrows at high speed in search of food. The tunnels Onix leaves are used as homes by Diglett.",
        "type": "Rock/Ground"
    }
]

#### Preparing Documents for Indexing

Splitting documents into smaller manageable chunks is ideal for efficient Elasticsearch indexing and search, especially for larger descriptions.

The following code prepares the documents for efficient processing by splitting them into manageable chunks while preserving metadata. It first extracts the descriptions and metadata, then uses `RecursiveCharacterTextSplitter` to break each description into overlapping segments. The result is a collection of smaller, context-rich text chunks, each paired with relevant metadata, ready for indexing.

In [21]:
metadata = []
content = []

for doc in data:
    content.append(doc["description"])
    metadata.append({"name": doc["name"], "type": doc["type"]})

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=512, chunk_overlap=256
)
docs = text_splitter.create_documents(content, metadatas=metadata)

#### Create Embeddings of the Data

Embeddings are numerical representations (vectors) of text. Text with similar meaning will have similar embedding vectors. You'll use an embedding model to create the embedding vectors of the data.

Initialize the embeddings using the `sentence-transformers/all-MiniLM-L12-v2` HuggingFace embedding model. You specify the `device` as `cuda` to utilize the GPU for faster computations and any additional paramaters like `normalize_embeddings` can be customized too.


In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L12-v2",
    model_kwargs={'device': 'cuda'},
    encode_kwargs={'normalize_embeddings': False}
)

#### Store the Embeddings in the Vector Store

Set up the `ElasticsearchStore` index using the `langchain-elasticsearch` integration. You'll also use the same index while querying the vector store using the RAG later.

Run the following cell to start indexing the documents.

In [23]:
index_name = "es-rag-pokemon"

documents = ElasticsearchStore.from_documents(
    docs,
    embeddings,
    index_name=index_name,
    es_url=es_url
)

#### Create a Retriever

Next, you'll create a retriever to fetch relevant documents based on user queries. You'll design a hybrid search query for Elasticsearch that combines both traditional keyword-based search and vector similarity search.

* [**Keyword Matching using BM25**](https://www.elastic.co/blog/practical-bm25-part-2-the-bm25-algorithm-and-its-variables): Ensures that documents containing the exact or similar terms (with fuzziness) are retrieved. This is useful for precise term matching and accommodates typos.

* [**Vector Similarity (KNN)**](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm): Retrieves documents that are semantically similar to the search query, even if they don't contain the exact search terms. This captures the meaning behind the words.


So, let's define a hybrid query function that uses both techniques.


In [24]:
def hybrid_query(search_query: str) -> Dict:
    vector = embeddings.embed_query(search_query)
    return {
        # Keyword matching
        "query": {
            "match": {
                "text": {
                    "query": search_query,
                    # Keyword matching with typo tolerance
                    "fuzziness": "AUTO",
                }
            },
        },
        # K-Nearest Neighbors
        "knn": {
            # The default vector field name in LangChain is "vector"
            "field": "vector",
            "query_vector": vector,
            "k": 5,
            "num_candidates": 10,
        }
    }

Initialize the retriever.

In [25]:
retriever = ElasticsearchRetriever.from_es_params(
    index_name=index_name,
    body_func=hybrid_query,
    content_field="text",
    url=es_url,
)

Here, you'll test if the retriever is working as intended.

In [ ]:
retriever.invoke("Pikachu")

### Generation

Next, the Generation phase involves prompting the LLM for an answer when the user asks a question. The retriever you created in the previous stage will be used here to provide more context.

You'll perform the following steps in this stage:

* Load a predefined RAG prompt.
* Define a function to format retrieved documents.
* Handle Gemma formatting using a custom output parser
* Chain everything together

#### Load a predefined prompt

You will use a [RAG prompt template](https://www.google.com/url?q=https%3A%2F%2Fsmith.langchain.com%2Fhub%2Frlm%2Frag-prompt) from LangChain Hub for the predefined prompt. It is useful for chat, QA, or other applications that rely on passing context to an LLM.


In [27]:
prompt = hub.pull("rlm/rag-prompt")
print(f"Prompt:\n\n{prompt.messages[0].prompt.template}")

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Prompt:

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:


#### The formatted context

Define a function to format the retrieved documents.

In [28]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

#### Custom Gemma Output Parser
Since Gemma models use specific formatting, you'll create an output parser to extract the instruction-tuned model's response properly.

In [29]:
class GemmaOutputParser(BaseTransformOutputParser[str]):
    def parse(self, text: str) -> str:
        model_start_token = "<start_of_turn>model\n"
        idx = text.rfind(model_start_token)
        return text[idx + len(model_start_token):] if idx > -1 else text

#### Chain everything together

Finally, you'll set up the RAG chain that ties everything together while relying on LangChain's [LCEL paradigm](https://python.langchain.com/v0.1/docs/expression_language/why/).

[The prompt](https://smith.langchain.com/hub/rlm/rag-prompt) you're using expects an input that includes two keys: `context` and `question`. The user only provides the question, so you need to obtain and format the relevant context using our retriever. Here's how it works:

* **Retrieve Relevant Documents**: Use `retriever` to get the relevant documents based on the user's question.

* **Format the Documents**: The retrieved documents may not be in a format suitable for our prompt, so you'll use the `format_docs` function to combine and format these documents into a single coherent string. This formatted string becomes our `context`. The pipe symbol here (`|`) is used to chain `retriever` and `format_docs`, resulting in the **formatted context**.

* **Pass the Question**: Pass the **user's question** directly under the `question` key via `RunnablePassthrough`. This behaves almost like the identity function, except that `RunnablePassthrough` allows you to pass the user's question directly through the chain to the prompt and the model without any modification (or add more keys to the output via `RunnablePassthrough.assign`). To learn more about this, read [the official documentation](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.passthrough.RunnablePassthrough.html).

* **Fill the Prompt**: Combine both the **formatted context** and the **user's question** into a single input dictionary for the prompt. This dictionary fills in the placeholders (like `context` and `question`) with the actual values from the dictionary. This results in a complete prompt text that is ready to be sent to the language model.

* **Generate Answer with the LLM**: The filled prompt is then passed to the `llm` (the Gemma model via Ollama). The LLM processes the prompt and generates a response based on the provided context and question.

* **Parse the LLM's Output**: The raw output from the LLM might include control tokens or additional formatting specific to the Gemma model. You'll use the `GemmaOutputParser` to parse the LLM's output and extract the final answer.


In simple terms, you gather the context related to the question, format it appropriately, and then provide both the formatted context and the question to the model to generate an informed answer.


In [30]:
# Create an actual chain

rag_chain = (
    # First you need retrieve documents that are relevant to the
    # given query
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    # The `context` and `question` are then passed to the prompt
    | prompt
    # The whole prompt will all the information is passed the LLM
    | llm
    # The answer of the LLM is parsed by the class defined above
    | GemmaOutputParser()
)

### Try It Out!

Finally, let's ask some questions and see how the RAG chain performs.

In [34]:
question = "What happens when several Pikachu gather?"
answer = rag_chain.invoke(question)
print(answer)

When several Pikachu gather, their electricity could build and cause lightning storms.  The strength of this electrical buildup increases with the number of Pikachu present. 





In [32]:
question = "Name a few Pokémon that can fly"
answer = rag_chain.invoke(question)
print(answer)

Dragonite and Charizard can fly.  They are both powerful Pokémon known for their flying abilities. 



In [33]:
question = "What's a Pokémon that loves round objects?"
answer = rag_chain.invoke(question)
print(answer)

Meowth loves round objects.  It searches for dropped coins at night. 





Congratulations! You've successfully built a local Retrieval-Augmented Generation (RAG) application using the quantized Gemma 2 model (Gemma 2 9B IT), LangChain, Ollama, and Elasticsearch.

Feel free to explore further by:

- Adding more data to Elasticsearch.
- Experimenting with different Gemma models from the [Ollama Gemma 2 Model Catalog](https://ollama.com/library/gemma2).
- Tweaking model parameters for better performance.
- Check [Elastic Cloud deployment](https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud) out and learn how to obtain your Elastic credentials ([ELASTIC_CLOUD_ID](https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id) and [ELASTIC_API_KEY](https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key)) for setting up a cloud instance instead.

By following this tutorial, you're now equipped to build your own local RAG applications and explore the capabilities of Gemma models combined with LangChain and Elasticsearch.
